In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/home/sehan/sehan_workspace/llama-3.1-8b"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    dtype="auto",
)




/home/humane/miniconda3/envs/knowledge_conflict/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

KeyboardInterrupt



In [ ]:
# 간단한 텍스트 생성 테스트
prompt = "What is artificial intelligence?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

In [ ]:
# RAG를 위한 벡터 데이터베이스 설정 (ChromaDB 사용)
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 문서 로딩 (예시 - 실제 문서 경로로 변경 필요)
# loader = DirectoryLoader('./documents/', glob="**/*.txt")
# documents = loader.load()

# 예시 문서 생성
from langchain.schema import Document

documents = [
    Document(page_content="Paris is the capital of France. It is known for the Eiffel Tower.", metadata={"source": "geography"}),
    Document(page_content="Python is a popular programming language. It is widely used for AI and data science.", metadata={"source": "tech"}),
    Document(page_content="The Louvre Museum is located in Paris and houses the Mona Lisa.", metadata={"source": "culture"}),
]

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
splits = text_splitter.split_documents(documents)

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 벡터 데이터베이스 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./RAG/vectorDB"
)

print(f"벡터 데이터베이스 생성 완료! 총 {len(splits)}개의 문서 청크 저장됨")